In [31]:
%load_ext autoreload
%autoreload 2
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
from visiprog.metric_learn.lfda_visiprog import LFDA_VISIPROG

%matplotlib inline
import numpy as np
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

D = count_matrix(groups_VSP, raw_feature.shape[0])
clusters_VSP = spectralClusteringDBSCAN(D, threshold=0,eps=0.001)


INFO:visiprog.data:After removing non zero entries, N is 3708
INFO:visiprog.data:Number of disconnected elements is 161



In [6]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])
label_VSP_clusters = get_VSP_label(clusters_VSP, raw_feature.shape[0])

In [7]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [41]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

list_img = read_img_list()

n_cluster = 1

for i in range(10):
    X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
    train_test_split(raw_feature, X_STSIMM, label_material, label_VSP_clusters, \
                     test_size=0.2, stratify=label_material)
    
    exemplar_index = find_exemplars(X_STSIMM_train, Y_VSP_train, n_cluster)
    exemplar_X = X_STSIMM_train[exemplar_index]
    exemplar_Y = Y_mat_train[exemplar_index]
    
    res = leave_one_sample_out(exemplar_X, exemplar_Y, X_mat_test, Y_mat_test)
    accuracy_STSIMM.append(res['accuracy'])
    
    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_train = model.transform(X_train)
        X_mat_test = model.transform(X_test)
        
        exemplar_index = find_exemplars(X_mat_train, Y_VSP_train, n_cluster)
        exemplar_X = X_mat_train[exemplar_index]
        exemplar_Y = Y_mat_train[exemplar_index]
        
        res = leave_one_sample_out(exemplar_X, exemplar_Y, X_mat_test, Y_mat_test)
        
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        accuracy_mat.append(np.nan)
    
    model_vsp_original = LFDA()
    try:
        # remove unlabeled data
        idx_outlier = np.where(Y_VSP_train == 0)[0]
        X_train_without_outlier = X_train[~idx_outlier,:]
        Y_VSP_train_without_outlier = Y_VSP_train[~idx_outlier]
        model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)
        
        X_vsp_original_train = model_vsp_original.transform(X_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        
        exemplar_index = find_exemplars(X_vsp_original_train, Y_VSP_train, n_cluster)
        exemplar_X = X_vsp_original_train[exemplar_index]
        exemplar_Y = Y_mat_train[exemplar_index]
        
        res_vsp_original = leave_one_sample_out(exemplar_X, exemplar_Y, X_vsp_original_test, Y_mat_test)
        
        print(res_vsp_original['accuracy'])
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        print(e)
        accuracy_vsp_original.append(np.nan)


    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_train = model_vsp.transform(X_train)
        X_vsp_test = model_vsp.transform(X_test)
        
        exemplar_index = find_exemplars(X_vsp_train, Y_VSP_train, n_cluster)      
        exemplar_X = X_vsp_train[exemplar_index]
        exemplar_Y = Y_mat_train[exemplar_index]
        
        res_vsp = leave_one_sample_out(exemplar_X, exemplar_Y, X_vsp_test, Y_mat_test)
       
        
        # remove trained data which are not labeled by VSP
#         indx = np.where(Y_VSP_train != 0)[0]
#         X_vsp_train = model_vsp.transform(X_train[indx])
#         res_vsp = kmean_subclass(X_vsp_train, Y_mat_train[indx], X_vsp_test, Y_mat_test, n_cluster)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(np.nan)
        

df = pd.DataFrame({'STSIM-M':accuracy_STSIMM,'visiprog':accuracy_vsp, \
                   'visiprog_original': accuracy_vsp_original, 'material':accuracy_mat}, \
                  columns=['STSIM-M','visiprog_original','visiprog','material'])
df.loc['average'] = df.mean()
display(df)


df.to_csv('thrasos/within_traincluster_exemlparcluster_k{}.csv'.format(n_cluster))



INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[11 29  7 ..., 50 58 41]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
0.67111534795
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[39 14 26 ..., 51 40 43]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
0.720686367969
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[24 57  1 ..., 50 13 18]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:Nearest neighbor on separate test data
INFO:visiprog.evaluate:Nearest neighbor on separate test data
0.743565300286
INFO:visiprog.evaluate:Nearest neighbor on separate test data
IN

,STSIM-M,visiprog_original,visiprog,material
0,0.819828,0.671115,0.807436,0.820782
1,0.841754,0.720686,0.818875,0.832221
2,0.834128,0.743565,0.806482,0.832221
3,0.822688,0.734032,0.808389,0.832221
4,0.815062,0.668255,0.808389,0.829361
5,0.831268,0.698761,0.841754,0.816015
6,0.845567,0.662536,0.828408,0.822688
7,0.818875,0.729266,0.818875,0.814109
8,0.830315,0.680648,0.815062,0.832221
9,0.820782,0.740705,0.799809,0.811249
